# Лабораторная работа №3
## Выполнил студент группы Казыханов Рамиль Ильясович БВТ2004

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [35]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

# Нужен для того, чтобы выделять найденную подстроку цветом
def print_found(src: str, start: int, length: int):
    if start != -1:
        print(src[0:start] + f'\033[1m\033[91m{src[start:start + length]}\033[0m' + src[start + length::])
    else:
        print(src)

# Ввод исходных данных
src_str = input('Исходная строка: ')
sub_str = input('Подстрока для поиска: ')
case_sensetive = int(input('Чувствителен к регистру? (1 - да, 0 - нет): ')) != 0

Исходная строка: i n t R_e smth blabla
Подстрока для поиска: R_e sm
Чувствителен к регистру? (1 - да, 0 - нет): 1


### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Стандартные методы

In [36]:
src = src_str
sub = sub_str

if not case_sensetive:
    src = src.lower()
    sub = sub.lower()
    

start_time = t.now().timestamp()
found_index = src.find(sub)
print("Время поиска: {:.6f} ms".format((t.now().timestamp() - start_time) * 1000))
print_found(src_str, found_index, len(sub_str))

Время поиска: 0.128984 ms
i n t R_e smth blabla


#### Алгоритм Кнута-Морриса-Пратта

In [37]:
def prefix_func(src: str) -> list[int]:
    result = [0] * len(src)
    
    i = 1
    j = 0
    
    while i < len(src):
        if src[i] == src[j]:
            result[i] = j + 1
            i += 1
            j += 1
        elif j == 0:
            result[i] = 0
            i += 1
        else:
            j = result[j - 1]
    
    return result

def kmp_find(src: str, sub_str: str, ignore_case = False) -> int:
    if len(sub_str) > len(src):
        return -1
    
    if ignore_case:
        src = src.lower()
        sub_str = sub_str.lower()
    
    p = prefix_func(sub_str)
    
    i = 0
    j = 0
    
    while i < len(src):
        if src[i] == sub_str[j]:
            i += 1
            j += 1
            
            if j == len(sub_str) - 1:
                return i + 1 - len(sub_str)
        elif j == 0:
            i += 1
            
            if i == len(src) - 1:
                return -1
        else:
            j = p[j - 1]
            
    return -1

start_time = t.now().timestamp()
found_index = kmp_find(src_str, sub_str, ignore_case = not case_sensetive)
print("Время поиска: {:.6f} ms".format((t.now().timestamp() - start_time) * 1000))
print_found(src_str, found_index, len(sub_str))

Время поиска: 0.102997 ms
i n t R_e smth blabla


#### Упрощенный алгоритм Бойера-Мура

In [91]:
def shifts_table(src: str) -> dict:
    table = {}
    N = len(src)
    
    for i in range(N - 2, -1, -1):
        symbol = src[i]
        if table.get(symbol) == None:
            table[symbol] = N - 1 - i
    
    last_symbol = src[-1]
    if table.get(last_symbol) == None:
        table[last_symbol] = N
        
    return table

def boyer_moore_find(src: str, sub_str: str, ignore_case = False):
    N = len(sub_str)
    
    if N > len(src):
        return -1
    
    if ignore_case:
        src = src.lower()
        sub_str = sub_str.lower()
    
    shifts = shifts_table(sub_str)
    
    i = N - 1
    
    while i < len(src):
        start = i + 1 - N
        
        if src[start: i + 1] == sub_str:
            return start
        
        symbol = src[i]
        shift = shifts.get(symbol)
        
        if shift == None:
            i += N
        else:
            i += shift
            
    return -1

start_time = t.now().timestamp()
found_index = boyer_moore_find(src_str, sub_str, ignore_case = not case_sensetive)
print("Время поиска: {:.6f} ms".format((t.now().timestamp() - start_time) * 1000))
print_found(src_str, found_index, len(sub_str))

Время поиска: 0.146151 ms
i n t R_e smth blabla


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [2]:
import copy
import math
from heapq import *


def swap(values: list, i: int, j: int):
    values[i], values[j] = values[j], values[i]


class Heap(object):
    def __init__(self):
        self.index = 0
        self.__data = []

    def push(self, item, priority: int):
        heappush(self.__data, (priority, self.index, item))
        self.index += 1

    def pop(self):
        return heappop(self.__data)[2]

    def size(self) -> int:
        return len(self.__data)

    def __str__(self) -> str:
        return str(self.__data)

    def __repr__(self) -> str:
        return self.__str__()


class Board:
    def __init__(self, tiles: list, move=None):
        self.tiles = tiles
        self.move = move

        size = math.sqrt(len(tiles))

        if not size.is_integer():
            raise ValueError('Invalid tiles size!')

        self.size = int(size)

    def neighbors(self) -> list:
        neighbors = []
        empty_index = self.empty_index()
        empty_row_index = empty_index // self.size

        if (empty_index - 1) // self.size == empty_row_index:
            new_tiles = copy.deepcopy(self.tiles)
            swap(new_tiles, empty_index, empty_index - 1)
            board = Board(new_tiles, self.tiles[empty_index - 1])
            neighbors.append(board)

        if (empty_index + 1) // self.size == empty_row_index:
            new_tiles = copy.deepcopy(self.tiles)
            swap(new_tiles, empty_index, empty_index + 1)
            board = Board(new_tiles, self.tiles[empty_index + 1])
            neighbors.append(board)

        if empty_index - 4 > 0:
            new_tiles = copy.deepcopy(self.tiles)
            swap(new_tiles, empty_index, empty_index - 4)
            board = Board(new_tiles, self.tiles[empty_index - 4])
            neighbors.append(board)

        if empty_index + 4 < len(self.tiles):
            new_tiles = copy.deepcopy(self.tiles)
            swap(new_tiles, empty_index, empty_index + 4)
            board = Board(new_tiles, self.tiles[empty_index + 4])
            neighbors.append(board)

        return neighbors

    def empty_index(self) -> int:
        for i in range(len(self.tiles)):
            if self.tiles[i] == 0:
                return i

        return -1

    def __str__(self) -> str:
        result = ''

        for i in range(self.size):
            for j in range(self.size):
                result += '{:4}'.format(self.tiles[i * self.size + j])
            result += '\n'

        return result

    def __repr__(self) -> str:
        return str(self.tiles)


class SlidePuzzleSolver:
    def __init__(self, n: int):
        self.__n = n
        self.__goal_tiles = []

        for i in range(n * n - 1):
            self.__goal_tiles.append(i + 1)

        self.__goal_tiles.append(0)

    def solve(self, initial_board: Board) -> list:
        if initial_board.size != self.__n:
            raise ValueError('Invalid tiles size!')

        if not self.is_solvable(initial_board):
            return []

        key = str(initial_board.tiles)
        came_from = {key: None}
        cost_so_far = {key: 0}

        heap = Heap()
        heap.push(initial_board, 0)

        current_board = None

        while heap.size() > 0:
            current_board = heap.pop()
            current_key = str(current_board.tiles)

            if self.is_goal(current_board):
                break

            for next_board in current_board.neighbors():
                next_key = str(next_board.tiles)
                new_cost = cost_so_far[current_key]
                if next_key not in cost_so_far or new_cost < cost_so_far[next_key]:
                    cost_so_far[next_key] = new_cost
                    priority = new_cost + self.manhattan(next_board)
                    heap.push(next_board, priority)
                    came_from[next_key] = current_board

        solution = []

        while current_board is not None and current_board.move is not None:
            solution.append(current_board.move)
            current_board = came_from[str(current_board.tiles)]

        solution.reverse()
        return solution

    def is_goal(self, board: Board) -> bool:
        return board.tiles == self.__goal_tiles

    def manhattan(self, board: Board) -> int:
        manhattan = 0

        for i in range(len(board.tiles)):
            num = board.tiles[i]

            if num != self.__goal_tiles[i] and num != 0:
                row = i // self.__n
                col = i % self.__n

                goal_index = 0

                for j in range(len(self.__goal_tiles)):
                    if self.__goal_tiles[j] == num:
                        goal_index = j
                        break

                goal_row = goal_index // self.__n
                goal_col = goal_index % self.__n

                manhattan += abs(goal_row - row) + abs(goal_col - col)

        return manhattan

    def is_solvable(self, board: Board) -> bool:
        inversions_count = 0

        for i in range(len(board.tiles) - 1):
            for j in range(i + 1, len(board.tiles)):
                curr_tile = board.tiles[i]
                next_tile = board.tiles[j]
                if curr_tile != 0 and next_tile != 0 and curr_tile > next_tile:
                    inversions_count += 1

        if self.__n % 2 != 0:
            return inversions_count % 2 == 0
        else:
            empty_index = board.empty_index()
            empty_row_index = empty_index // self.__n
            return (inversions_count + empty_row_index) % 2 != 0


def main():
    # [11, 6, 0, 8, 15, 4, 12, 7, 5, 9, 3, 2, 1, 14, 10, 13]
    # [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]
    # [4, 5, 6, 3, 1, 0, 11, 12, 9, 2, 13, 15, 14, 10, 7, 8]
    initial_board = Board([1, 2, 4, 0, 9, 3, 11, 8, 7, 13, 10, 14, 12, 5, 6, 15])
    solver = SlidePuzzleSolver(4)

    print(solver.is_solvable(initial_board))
    print(solver.solve(initial_board))


main()

True
[4, 2, 3, 9, 7, 13, 10, 6, 15, 14, 6, 11, 9, 7, 13, 12, 5, 10, 12, 13, 7, 9, 11, 12, 9, 7, 13, 9, 10, 15, 14, 6, 12, 11, 7, 10, 15, 14, 6, 12, 11, 15, 14, 6, 15, 11, 12, 15, 6, 14, 11, 6, 15, 12, 8, 7, 6, 11, 10, 6, 7, 8, 12, 15, 14, 5, 9, 13, 6, 10, 5, 9, 13, 5, 10, 6, 5, 10, 9, 14, 11, 9, 10, 13, 14, 10, 9, 11, 10, 14, 13, 9, 11, 10, 15, 12, 10, 11, 14, 15, 11, 10, 12, 11, 15, 14, 10, 12, 11, 15, 12, 11, 8, 7, 11, 12, 15, 8, 12, 11, 2, 3, 6, 2, 7, 4, 3, 6, 2, 7, 6, 3, 4, 12, 8, 15, 11, 8, 12, 6, 8, 11, 15, 12, 6, 8, 11, 6, 12, 15, 14, 10, 6, 11, 7, 6, 10, 14, 15]


### Вывод

В ходе проделанной лабораторной работы можно сделать вывод о том, что наиболее эффективным из рассмотренных вариантов поиска подстроки в строке является алгоритм Кнута-Морриса-Пратта, хоть и для его реалзиации нужно приложить больше усилий. Что касается второго задания: на сегодняшний день наиболее эффективными решениями подобных задач являются алгоритмы поиска пути, один из которых, так называемый алгоритм поиска пути A*, был рассмотрен в данной лабораторной работе.